In [ ]:
import sys
sys.path.append('/opt/deep_learning/utilities')

from spark_setup import SetupEnvironment
environment = SetupEnvironment (
    
                    conda_env = 'my_env', # should match with jupyter kernel used (e.g. my_env or deep_Learning)

                    project_name = 'NoLivyExampleProject', # defaults to Domino project name if not set

                    additional_settings= {

                                #"spark.executor.memory" : "130G",
                                "spark.driver.memory": "130G", #THIS NEEDS TO BE SET FOR 3.1 TO HAVE ADEQUATE SPACE TO READ D8 tO PANDAS
                                #"spark.executor.cores": "2",
                                #cores max == 24 instances * 20 each(large) * 0.8 = rough384
                                #"spark.cores.max": "180",
                                #"spark.driver.memoryOverhead":"20G", 
                                #"spark.executor.memoryOverhead": "20G",
                                #"spark.sql.autoBroadcastJoinThrishold":"-1",
                                #"spark.sql.shuffle.partitions": "200", 
                                #'spark.shuffle.service.enabled' : 'true',
                                #'spark.excludeOnFailure.enabled' : 'true',
                                #'spark.excludeOnFailure.application.fetchFailure,enabled';'true' 
                                #"spark.executorEnv.PYARROW_IGNORE_TIMEZONE":"1", 
                                #"spark.sql.execution.arrow.pyspark.enabled";'true', 
                                #"spark.sql.execution.arrow.pyspark.fallback.enabled":'true'
                    } # additional spark configuration (optional)
            )
spark = environment.setup_spark()
dcRead = environment.setup_DataCatalog()
s3 = environment.setup_s3()

### Import Functions

In [ ]:
import s3fs
import pickle 
import pyarrow 
import os 
import sys
import pandas as pd 
import datetime 
import numpy as np
from pyarrow import parquet as pa 
import ison
from tensorflow import keras 
from tensorflow.keras import layers 
from matplotlib import pyplot as plt 
import pyspark.sql.functions as F
pd.set_option('display.max_rows', None)

In [ ]:
# vizualization modules
from plotly.offline import download_plotlyjs, plot, iplot 
import plotly.graph_objs as go 
import plotly.io as pio 
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
from keras.models import load_model

In [ ]:
def read_pipeline_parquet(s3a_bucket_str, dataframe_name, date_start, date_end=None, verbose=False, version=None):

    #reads loaded data into parquet for future consumption 
    
    if date_start == None:
        file_path = s3a_bucket_str+dataframe_name 
    elif date_end != None:
        file_path = s3a_bucket_str+dataframe_name+'_{}'.format(date_start)+'_{}'.format(date_end)
    else:
        file_path = s3a_bucket_str+dataframe_name+'_{}'.format(date_start)

    if verbose==True:
        print(f'Reading from {file_patht}')
    else:
        pass
    if version == None:
        pass
    else:
        file_path =file_path +'_v'+str(version)
    new_file = spark.read.parquet(file_path)
    return new_file

In [ ]:
#For Reading/Writing
PROBE DATE = '2022-01-20'
CURRENT_VERSION = 0
BUCKET_STRING= 's3a://9262-4648-2427-mrp-cat-kms/users/K30577/'
RANDOM_SEED = 42
TRAIN_PROPORTION = 0.75
#SLICES = 6 #(chosen in last step of process)

### Load Data Samples for Testing/Analysis

In [ ]:
#Sampling Method 1

Final_IV_Data_Sample = read_pipeline_parquet(BUCKET_STRING, "actrain9_IV_normalized_rows_100_1ea",PROBE_DATE, version=CURRENT_VERSION)

In [ ]:
Final_IV_Data_Sample

In [ ]:
#Sampling Method 2

Final_IV_Data_Sample = read_pipeline_parquet(BUCKET_STRING,"actrain9_IV_normalized_rows_005", PROBE_DATE, version=CURRENT_VERSION)

In [ ]:
Final_IV_Data_Sample

### Convert to Pandas, Clean Data, Split into Test and Valdiation Sets

In [ ]:
Final_IV_Data_Sample_Pre_PD = Final_IV_Data_Sample.select('chain_id', 'all_items')

In [ ]:
Final_IV_Data_Sample_PD = Final_IV_Data_Sample_Pre_PD.toPandas()

In [ ]:
Final_IV_Data_Sample_PD.describe()

In [ ]:
pd.DataFrame(Final_IV_Data_Sample_PD.isna().sum()[Final_IV_Data_Sample_PD.isna().sum() > 0], columns = ['count nan values'])

In [ ]:
Final_IV_Data_Sample_PD.info()

In [ ]:
Final_IV_Data_Sample_PD['np_items'] = Final_IV_Data_Sample_PD['all_items'].to_numpy()
Final_IV_Data_Sample_PD['np_items_1st'] = Final_IV_Data_Sample_PD['np_items'].str[0]
Final_IV_Data_Sample_PD_DeNan = Final_IVIData_Sample_PD[Final_IV_Data_Sample_PD['np_items_1st'].isna()==False]

In [ ]:
Final_IV_Data_Sample_PD_DeNan.describe()

In [ ]:
pd.DataFrame(Final_IV_Data_Sample_PD_DeNan.isna().sum()[Final_IV_Data_Sample_PD_DeNan.isna().sum() > 0], columns = ['count nan values'])

In [ ]:
Final_IV_Data_Sample_PD_DeNan.info()

In [ ]:
num_events = len(Final_IV_Data_Sample_PD_DeNan)

In [ ]:
train_val_df = Final_TV_Data_Sample_PD_DeNan.sample(frac=1, random_state = RANDOM_SEED).reset_index()
train_df = train_val_df.loc[: int (TRAIN_PROPORTION*num_events)]
val_df = train_val_df.loc[int (TRAIN_PROPORTION*num_events)+1:]


In [ ]:
X_train = np.stack(train_df['all_items'].to_numpy(), axis=0)
X_train = X_train.reshape(*(list(X_train.shape)+[1]))

In [ ]:
X_train

In [ ]:
X_val = np.stack(vat_df['all_items'].to_numpy(), axis=0)
X_val = x_val.reshape(*(list(X_val.shape)+[1]))

In [ ]:
X_val

### Load Retraining Candidate Models (AC1) and Prototype Model (ACO)

In [ ]:
AC0 = load_model('AC0')

In [ ]:
AC0.summary()

In [ ]:
AC1_1 =Load_model('autoencoder_retrain_1')
AC1_2 = load_mode ('autoencoder_retrain_2')
AC1_3 = load_model('autoencoder_retrain_3')


In [ ]:
AC1_1.summary()

In [ ]:
AC1_2.summary()

In [ ]:
AC1_3.summary()

### Generate Model Predictions for Validation Set and MAE Scores for Individual Observations (SubWindows)

In [ ]:
AC0_Pred = AC0.predict(X_val)
AC0_mae_loss = np.mean(np.abs(AC0_Pred - X_val), axis=1).flatten()

In [ ]:
AC1_1_Pred = AC1_1.predict(X_val)
AC1_1_mae_loss = np.mean(np.abs (AC1_1_Pred - X_val), axis=1).flatten()


In [ ]:
AC1_2_Pred = AC1_2.predict(X_val)
AC1_2_mae_loss = np.mean(np.abs(AC1_2_Pred - X_val), axis=1).flatten()

In [ ]:
AC1_3_Pred = AC1_3.predict(X_ val)
AC1_3_mae_loss = np.mean(np.abs(AC1_3_Pred - X_val), axis=1).flatten()

### Create Pandas Data Frame With MAE Scores by Observation

In [ ]:
MAE_Comparison = pd.DataFrame({'AC0_MAE' :AC0_mae_Loss, 'AC1_1_MAE': AC1_1_mae_loss, 'AC1_2_MAE':AC1_2_mae_Loss, 'AC1_3_MAE' :AC1_3_mae_loss})

In [ ]:
MAE_Comparison

### Look at Scatterplot of Two Sets of Observations for Subset of Data

In [ ]:
SAMPLE_SIZE = 10000
MAE_Comparison_Sample = MAE_Comparison.sample(SAMPLE_SIZE)
fig = go.Figure(data=go.Scatter(x=MAE_Comparison_Sample['AC1_2_MAE'], y=MAE_Comparison_Sample['AC0_MAE'], mode = 'markers'))
fig.update_yaxes(title = 'Old AC Score')
fig.update_xaxes(title = 'New AC Score')
fig.show()

### Look at Individual Trace for Randomly Chosen Rows

In [ ]:
RandomRow = MAE_Comparison.sample(1)

Random_AC1_3 = RandomRow['AC1_3_MAE'].iloc(0)[0]
Random_AC1_2 = RandomRow['AC1_2_MAE'].iloc(0)[0]
Random_AC1_1 = RandomRow[' AC1_1_MAE'].iloc(0)[0]
Random_AC0 = RandomRow['ACO_MAE'].iloc(0)[0]
#Random_ACNew = RandomRow['ACNew_Mae'].iloc(0)[0]
RandomCase = RandomRow.index[0]
Case = RandomCase
Actuals = X_val[Case]
AC0s = AC0_Pred[Case]
AC1_1s = AC1_1_Pred[Case]
AC1_2s = AC1_2_Pred[Case]
AC1_3s = AC1_3_Pred[Case]
ActualsArray = []
AC0sArray= []
AC1_1sArray= []
AC1_2sArray= []
AC1_3sArray= []
StepsArray = []
for x in range (0,120):
    ActualsArray.append(Actuals[x][0])
    AC0sArray.append(AC0s[x][0])
    AC1_1sArray.append(AC1_1s[x][0])
    AC1_2sArray.append(AC1_2s[x][1])
    AC1_3sArray.append(AC1_3s[x][0])
    StepsArray.append(x+1)
fig = go.Figure()  
# Add traces
fig.add_trace(
    go.Scatter(x=StepsArray, y=ActualsArray, name="Actuals", mode= 'lines+markers'), 
              )
fig.add_trace(
    go.Scatter(x=StepsArray, y=Ac0sArray, name="AC0s", mode= "lines+markers"),
             )
fig.add_trace(
    go.Scatter(x=StepsArray, y=AC1_1sArray, name="AC1_1s", mode= 'lines+markers'),
            )
fig.add_trace(
    go.Scatter(x=StepsArray, y = AC1_2sArray, name="AC1_2s", mode= 'lines+markers') , 
            )
fig.add_trace(
    go.Scatter(x=StepsArray, y = AC1_3sArray, name="AC1_3s", mode= 'lines+markers') , 
            )

#fig.update_yaxes(range= [0,1])
fig.update_layout(title = 'AC_0 MAE:'+str(Random_AC0) +'   \t AC_1_1 MAE:'+str(Random_AC1_1)+'   \t AC_1_2 MAE:'+str(Random_AC1_2)+'   \t AC_1_3 MAE:'+str(Random_AC1_3))
fig.show()


### Look at Cross Tabs of Original AC and Candidate Replacements

In [ ]:
MAE_CUTOFF = 0.015

In [ ]:
MAE_Comparison['AC0_Pos'] = MAE_Comparison['AC0_MAE'] > MAE_CUTOFF
MAE Comparison['AC1_1_Pos'] = MAE_Comparison['AC1_1_MAE'] > MAE_CUTOFF
MAE_Comparison['AC1_2_Pos'] = MAE_Comparison['AC1_2 MAE'] > MAE_CUTOFF
MAE_Comparison['AC1_3_Pos'] = MAE_Comparison['AC1_3_MAE'] > NAE_CUTOFF


In [ ]:
pd.crosstab(MAE_Comparison['AC0_Pos'] ,MAE_Comparison['AC1_1 _Pos'])

In [ ]:
pd.crosstab(MAE_Comparison["AC0_Pos"],MAE_Comparison['AC1_2_Pos'])

In [ ]:
pd.crosstab(MAE_Comparison['AC0_Pos'] ,MAE_Comparison['AC1_3_Pos'])

### Save favored model as AC1

In [ ]:
#AC1_2.save("AC1")

In [ ]:
model = load_model('AC0')

In [ ]:
model_ts = datetime.datetime.now()
EPOCHS = 10
BATCH_SIZE = 2048*2

history = model.fit(

    X_train, 
    X_train, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    validation_data=(X_val,X_val), 
    callbacks=[
            keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, mode ="min")
            ]
)

In [1]:
### validation

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import keras
import tensorflow as tf
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from scipy.stats import chi2_contingency
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.style.use('dark_background')
import warnings
warnings.filterwarnings('ignore')


In [ ]:
plt.plot(history.history['loss'], c = 'b', label='train')
plt.plot(history.history['val_loss'], c = 'y', label='validation')
plt.legend()
plt.title("Reconstruction Loss ")
plt.xlabel("Epochs")
plt.ylabel("MAE")
plt.show()

In [ ]:
mets = []
train = []
val = []

mets.append('mean_squared_error')
train.append(mean_squared_error(x_train, x_train_pred))
val.append(mean_squared_error(x_val, x_val_pred))

mets.append('mean_absolute_error')
train.append(mean_absolute_error(x_train, x_train_pred))
val.append(mean_absolute_error(x_val, x_val_pred))

mets.append('r2_multi_output_regres')
train.append(MultiOutputRegressor(Ridge(random_state=123)).fit(x_train, x_train_pred).score(x_train, x_train_pred))
val.append(MultiOutputRegressor(Ridge(random_state=123)).fit(x_val, x_val_pred).score(x_val, x_val_pred))

mets.append('r2_sklearn')
train.append(r2_score(x_train, x_train_pred))
val.append(r2_score(x_val, x_val_pred))

metrics = pd.DataFrame({'metrics' : mets, 'train_set' : train, 'validation_set' : val })
metrics

In [ ]:
data = [x_train.tolist(), x_train_pred.tolist()]
chi_square_dict = {'stats':chi2_contingency(data)[0], 'p':chi2_contingency(data)[1],'dof':chi2_contingency(data)[2]}
print(f'train_chi2_contingency: \n {chi_square_dict}')

In [ ]:
data = [x_val.tolist(), x_val_pred.tolist()]
chi_square_dict = {'stats':chi2_contingency(data)[0], 'p':chi2_contingency(data)[1],'dof':chi2_contingency(data)[2]}
print(f'validation_chi2_contingency: \n {chi_square_dict}')